# Capstone Project

## 1. Ingesting data

In [2]:
import pandas as pd
import numpy as np
import time
import logging
import os
import sys

In [4]:
t = time.time()
auth_user = pd.read_csv('data/ebdb_public_auth_user.csv')
print("auth_user loaded: %d lines in %f seconds" % (auth_user.shape[0], time.time() - t))

auth_user loaded: 71638 lines in 0.354677 seconds


In [5]:
t = time.time()
payment_app_product = pd.read_csv('data/ebdb_public_payment_app_product.csv')
print("payment_app_product loaded: %d lines in %f seconds" % (payment_app_product.shape[0], time.time() - t))

payment_app_product loaded: 29 lines in 0.003525 seconds


In [6]:
t = time.time()
payment_app_subscription = pd.read_csv('data/ebdb_public_payment_app_historicalsubscription.csv')
print("payment_app_subscription loaded: %d lines in %f seconds" % (payment_app_subscription.shape[0], time.time() - t))

payment_app_subscription loaded: 42172 lines in 0.406074 seconds


/Users/carlos/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (9,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
t = time.time()
frontend_brazil_pages = pd.read_csv('data/analytics_frontend_brazil_pages.csv')
print("frontend_brazil_pages loaded: %d lines in %f seconds" % (frontend_brazil_pages.shape[0], time.time() - t))

/Users/carlos/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (28,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


frontend_brazil_pages loaded: 5747993 lines in 116.072547 seconds


In [8]:
t = time.time()
frontend_brazil_identifies = pd.read_csv('data/analytics_frontend_brazil_identifies.csv')
print("frontend_brazil_identifies loaded: %d lines in %f seconds" % (frontend_brazil_identifies.shape[0], time.time() - t))

frontend_brazil_identifies loaded: 141357 lines in 1.873921 seconds


/Users/carlos/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (25,26,27,28,29,30,31,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
t = time.time()
frontend_brazil_tracks = pd.read_csv('data/analytics_frontend_brazil_tracks.csv')
print("frontend_brazil_tracks loaded: %d lines in %f seconds" % (frontend_brazil_tracks.shape[0], time.time() - t))

/Users/carlos/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (20,26,27,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


frontend_brazil_tracks loaded: 1343225 lines in 16.996525 seconds


In [10]:
t = time.time()
brazil_events_signup = pd.read_csv('data/analytics_brazil_events_event_sign_up.csv')
print("brazil_events_signup loaded: %d lines in %f seconds" % (brazil_events_signup.shape[0], time.time() - t))

brazil_events_signup loaded: 31030 lines in 0.656590 seconds


/Users/carlos/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (25,27,29,31,33,34,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
t = time.time()
analytics_tables_course_enrollments = pd.read_csv('data/analytics_analytics_tables_course_enrollments_br.csv')
print("analytics_tables_course_enrollments loaded: %d lines in %f seconds" % (analytics_tables_course_enrollments.shape[0], time.time() - t))

analytics_tables_course_enrollments loaded: 187952 lines in 0.731893 seconds


In [12]:
t = time.time()
zendesk_data = pd.read_csv('data/zendesk_export.csv')
print("zendesk_data loaded: %d lines in %f seconds" % (zendesk_data.shape[0], time.time() - t))

zendesk_data loaded: 11829 lines in 0.127155 seconds


## 2. Preprocess Features

### 2.1. First step: *email*, *is_paying_student* and *register_date*

In [70]:
paying_students = payment_app_subscription.loc[lambda df: df['status'].isin(['active', 'payment_credit_retry'])]
paying_students.shape

(18692, 43)

In [74]:
paying_students = paying_students.groupby('user_id').first()[['id', 'register_date']]
paying_students.reset_index(level=0, inplace=True)
paying_students.head()

,user_id,id,register_date
0,7,670,2016-08-16 18:32:12.533034
1,13,1482,2016-10-08 23:05:24.148811
2,20,3436,2017-01-06 22:07:58.437857 -02:00
3,28,1656,2016-10-20 11:32:45.037242 -02:00
4,41,1397,2016-10-03 20:41:56.352383


In [76]:
all_users = auth_user[['id', 'email', 'first_name', 'last_name', 'date_joined']]
all_users.columns = ['user_id', 'email', 'first_name', 'last_name', 'date_joined']
all_users.head()

,user_id,email,first_name,last_name,date_joined
0,39,vinicius.zilinski@gmail.com,Vinicius,Zilinski,2016-06-23 18:10:54.299931
1,6,leeo.apc@gmail.com,Leo,A.c,2016-06-21 23:18:50.152776
2,8,sardinha.dias@gmail.com,Carlos,sardinha dias,2016-06-22 00:22:49.282586
3,10,thiagonv@gmail.com,Thiago,Vilela,2016-06-22 10:57:22.246017
4,11,ygorats@gmail.com,Ygor,Santos,2016-06-22 14:06:45.355805


In [99]:
def is_paying_student(row):
    if np.isnan(row['subscription_id']):
        return 0
    else:
        return 1

x = all_users.join(paying_students.set_index('user_id'), on='user_id')
x.rename(columns={'id': 'subscription_id'}, inplace=True)
x['is_paying_student'] = x.apply(lambda row: is_paying_student(row), axis=1)
x.head(10)

,user_id,email,first_name,last_name,date_joined,subscription_id,register_date,is_paying_student
0,39,vinicius.zilinski@gmail.com,Vinicius,Zilinski,2016-06-23 18:10:54.299931,NaN,NaN,0
1,6,leeo.apc@gmail.com,Leo,A.c,2016-06-21 23:18:50.152776,NaN,NaN,0
2,8,sardinha.dias@gmail.com,Carlos,sardinha dias,2016-06-22 00:22:49.282586,NaN,NaN,0
3,10,thiagonv@gmail.com,Thiago,Vilela,2016-06-22 10:57:22.246017,NaN,NaN,0
4,11,ygorats@gmail.com,Ygor,Santos,2016-06-22 14:06:45.355805,NaN,NaN,0
5,13,lazaro_neto_@hotmail.com,Lazaro Lacir,Moraes Neto,2016-06-22 15:41:57.398526,1482.0,2016-10-08 23:05:24.148811,1
6,15,jonatacw@gmail.com,Jonata,Wieczynski,2016-06-22 17:35:10.147516,NaN,NaN,0
7,17,laertejjunior@gmail.com,Laerte,Mercier,2016-06-22 18:06:27.395793,NaN,NaN,0
8,19,victormalagoni@hotmail.com,Victor,Pires Malagoni,2016-06-22 19:33:19.040524,NaN,NaN,0
9,20,a30673083@gmail.com,Jose,Ramirez,2016-06-22 21:50:31.777444,3436.0,2017-01-06 22:07:58.437857 -02:00,1


### 2.2. Webinar Enrollments

In [100]:
webinar_enrollments = brazil_events_signup[['email', 'enrollment_date']]
webinar_enrollments = webinar_enrollments.groupby('email').count()
webinar_enrollments.reset_index(level=0, inplace=True)
webinar_enrollments.rename(columns={'enrollment_date': 'webinar_enrollments'}, inplace=True)
webinar_enrollments.head(10)

,email,webinar_enrollments
0,0.adrian.axel@gmail.com,4
1,0matheus.araujo0@gmail.com,1
2,111@111.com,1
3,1162.eoj@gmail.com,1
4,130577er@gmail.com,1
5,146050u54@gmail.com,1
6,15cocosantos@gmail.com,1
7,15thlima@gmail.com,1
8,17salvador@gmail.com,5
9,1995.guilherme@gmail.com,1


In [103]:
x = x.join(webinar_enrollments.set_index('email'), on='email')

In [110]:
x['webinar_enrollments'] = x['webinar_enrollments'].fillna(0).astype(int)
x.head(10)

,user_id,email,first_name,last_name,date_joined,subscription_id,register_date,is_paying_student,webinar_enrollments
0,39,vinicius.zilinski@gmail.com,Vinicius,Zilinski,2016-06-23 18:10:54.299931,NaN,NaN,0,0
1,6,leeo.apc@gmail.com,Leo,A.c,2016-06-21 23:18:50.152776,NaN,NaN,0,1
2,8,sardinha.dias@gmail.com,Carlos,sardinha dias,2016-06-22 00:22:49.282586,NaN,NaN,0,0
3,10,thiagonv@gmail.com,Thiago,Vilela,2016-06-22 10:57:22.246017,NaN,NaN,0,0
4,11,ygorats@gmail.com,Ygor,Santos,2016-06-22 14:06:45.355805,NaN,NaN,0,1
5,13,lazaro_neto_@hotmail.com,Lazaro Lacir,Moraes Neto,2016-06-22 15:41:57.398526,1482.0,2016-10-08 23:05:24.148811,1,0
6,15,jonatacw@gmail.com,Jonata,Wieczynski,2016-06-22 17:35:10.147516,NaN,NaN,0,0
7,17,laertejjunior@gmail.com,Laerte,Mercier,2016-06-22 18:06:27.395793,NaN,NaN,0,0
8,19,victormalagoni@hotmail.com,Victor,Pires Malagoni,2016-06-22 19:33:19.040524,NaN,NaN,0,0
9,20,a30673083@gmail.com,Jose,Ramirez,2016-06-22 21:50:31.777444,3436.0,2017-01-06 22:07:58.437857 -02:00,1,1


### 2.3. Course Enrollments

In [125]:
course_enrollments = analytics_tables_course_enrollments[['user_id', 'course_key']]
course_enrollments = course_enrollments.groupby('user_id').count()
course_enrollments.reset_index(level=0, inplace=True)
course_enrollments = course_enrollments.join(auth_user[['username', 'email']].set_index('username'), on='user_id')
course_enrollments.rename(columns={'course_key': 'free_course_enrollments'}, inplace=True)
course_enrollments = course_enrollments[['email', 'free_course_enrollments']]
course_enrollments.head()

,email,free_course_enrollments
0,deivissondine@gmail.com,2
1,NaN,1
2,NaN,1
3,anajuliapc@gmail.com,4
4,NaN,1


In [126]:
x = x.join(course_enrollments.set_index('email'), on='email')
x['free_course_enrollments'] = x['free_course_enrollments'].fillna(0).astype(int)

,user_id,email,first_name,last_name,date_joined,subscription_id,register_date,is_paying_student,webinar_enrollments,free_course_enrollments
0,39,vinicius.zilinski@gmail.com,Vinicius,Zilinski,2016-06-23 18:10:54.299931,NaN,NaN,0,0,4
1,6,leeo.apc@gmail.com,Leo,A.c,2016-06-21 23:18:50.152776,NaN,NaN,0,1,20
2,8,sardinha.dias@gmail.com,Carlos,sardinha dias,2016-06-22 00:22:49.282586,NaN,NaN,0,0,5
3,10,thiagonv@gmail.com,Thiago,Vilela,2016-06-22 10:57:22.246017,NaN,NaN,0,0,12
4,11,ygorats@gmail.com,Ygor,Santos,2016-06-22 14:06:45.355805,NaN,NaN,0,1,1
5,13,lazaro_neto_@hotmail.com,Lazaro Lacir,Moraes Neto,2016-06-22 15:41:57.398526,1482.0,2016-10-08 23:05:24.148811,1,0,1
6,15,jonatacw@gmail.com,Jonata,Wieczynski,2016-06-22 17:35:10.147516,NaN,NaN,0,0,1
7,17,laertejjunior@gmail.com,Laerte,Mercier,2016-06-22 18:06:27.395793,NaN,NaN,0,0,4
8,19,victormalagoni@hotmail.com,Victor,Pires Malagoni,2016-06-22 19:33:19.040524,NaN,NaN,0,0,1
9,20,a30673083@gmail.com,Jose,Ramirez,2016-06-22 21:50:31.777444,3436.0,2017-01-06 22:07:58.437857 -02:00,1,1,3


### 2.4. Visits